# Use AutoAI and batch deployment to predict credit risk with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning Service. It introduces commands for getting data, training experiments, persisting pipelines, publishing models, deploying models and scoring.

Some familiarity with cURL is helpful. This notebook uses cURL examples.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train AutoAI models.
-  Downloading computed models to local storage.
-  Batch deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)  
2.	[Experiment definition](#experiment_definition)  
3.	[Experiment Run](#run)  
4.	[Historical runs](#runs)  
5.	[Deploy and Score](#deploy_and_score)  
6.	[Cleaning](#cleaning) 
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage can be found <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**

You can find your COS credentials in COS instance dashboard under the **Service credentials** tab.
Go to the **Endpoint** tab in the COS instance's dashboard to get the endpoint information.

Authenticate the Watson Machine Learning service on IBM Cloud.

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.  

**NOTE:** You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.  

**Action:** Fill below information to get started with this notebook.


In [ ]:
%env API_KEY=...
%env WML_ENDPOINT_URL=...
%env WML_INSTANCE_CRN="fill out only if you want to create a new space"
%env WML_INSTANCE_NAME=...

%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=...
%env COS_BUCKET=...
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=...

%env SPACE_ID="fill out only if you have space already created"

%env DATAPLATFORM_URL=https://api.dataplatform.cloud.ibm.com   
%env AUTH_ENDPOINT=https://iam.test.cloud.ibm.com/oidc/token

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [2]:
%%bash --out token

curl -sk -X POST \
    --header "Content-Type: application/x-www-form-urlencoded" \
    --header "Accept: application/json" \
    --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
    --data-urlencode "apikey=$API_KEY" \
    "$AUTH_ENDPOINT" \
    | cut -d '"' -f 4

In [3]:
%env TOKEN=$token 

env: TOKEN=eyJraWQiOiIyMDIwMDgyMjE2NTYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC01NTAwMDY3TkpEIiwiaWQiOiJJQk1pZC01NTAwMDY3TkpEIiwicmVhbG1pZCI6IklCTWlkIiwiaWRlbnRpZmllciI6IjU1MDAwNjdOSkQiLCJnaXZlbl9uYW1lIjoiSmFuIiwiZmFtaWx5X25hbWUiOiJTb2x0eXNpayIsIm5hbWUiOiJKYW4gU29sdHlzaWsiLCJlbWFpbCI6Ikphbi5Tb2x0eXNpa0BpYm0uY29tIiwic3ViIjoiSmFuLlNvbHR5c2lrQGlibS5jb20iLCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIxY2Y5ZjBlYjZhZjQ0NTlmOTQ3ODcxYmQzM2I0NzMxOCIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTU5ODM0NTU5NCwiZXhwIjoxNTk4MzQ5MTk0LCJpc3MiOiJodHRwczovL2lhbS50ZXN0LmNsb3VkLmlibS5jb20vb2lkYy90b2tlbiIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3BlIjoiaWJtIG9wZW5pZCIsImNsaWVudF9pZCI6ImRlZmF1bHQiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.cvzoMMOA9Lw2n9bsJWm1EdMQSedHKmoJah1YCE0i-AvbXyh4qHvv6zl9Etcsm06XNMVPWD6eS-G_DXwu1oO8wN94VKVX-TAKMswg05bxIXXe9KCM58DV8sDew9IwGGGajZexRC2cEhHnNG5F4vCAAzUq6bKEwL2LR1sEf91rBB39lxHfEh2fFywWvPj93VlR-_1NblQsew8olr8CffdOViq6VFipMj_0Vv5LYykf_vSoiPBiVzX25ZqvMnNrD2Hjh_TCpmzx

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

<a id="experiment_definition"></a>
## 2. Experiment / optimizer configuration

Provide input information for AutoAI experiment / optimizer:
- `name` - experiment name
- `learning_type` - type of the problem
- `label` - target column name
- `scorer_for_ranking` - optimization metric
- `holdout_param` - percentage of training data to use as a holdout [0 - 1]
- `daub_include_only_estimators` - list of estimators to use

You can modify `parameters` section of the following cURL call to change AutoAI experiment settings.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Pipelines/pipelines_create" 
target="_blank" rel="noopener no referrer">Define AutoAI experiment.</a>

In [4]:
%%bash --out pipeline_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data  '{"space_id": "'"$SPACE_ID"'", "name": "Credit Risk Prediction - AutoAI", "description": "", "document": {"doc_type": "pipeline", "version": "2.0", "pipelines": [{"id": "autoai", "runtime_ref": "hybrid", "nodes": [{"id": "automl", "type": "execution_node", "parameters": {"stage_flag": true, "output_logs": true, "input_file_separator": ",", "optimization": {"learning_type": "binary", "label": "Risk", "max_num_daub_ensembles": 1, "daub_include_only_estimators": ["ExtraTreesClassifierEstimator", "GradientBoostingClassifierEstimator", "LGBMClassifierEstimator", "LogisticRegressionEstimator", "RandomForestClassifierEstimator", "XGBClassifierEstimator", "DecisionTreeClassifierEstimator"], "scorer_for_ranking": "roc_auc", "compute_pipeline_notebooks_flag": true, "run_cognito_flag": true, "holdout_param": 0.1}}, "runtime_ref": "autoai", "op": "kube"}]}], "runtimes": [{"id": "autoai", "name": "auto_ai.kb", "app_data": {"wml_data": {"hardware_spec": { "name": "L"}}}, "version": "3.0.2"}],"primary_pipeline": "autoai"}}' \
    "$WML_ENDPOINT_URL/ml/v4/pipelines?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 5p

In [5]:
%env PIPELINE_ID=$pipeline_id

env: PIPELINE_ID=b7904fd9-6a26-4e5e-b6df-92722f8ee94e


<a id="experiment_details"></a>
### Get experiment details
To retrieve AutoAI experiment / optimizer configuration you can follow below cURL GET call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Pipelines/pipelines_get" 
target="_blank" rel="noopener no referrer">Get experiment / optimizer information</a>

In [6]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/pipelines/$PIPELINE_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "document": {
            "doc_type": "pipeline",
            "pipelines": [
                {
                    "id": "autoai",
                    "nodes": [
                        {
                            "id": "automl",
                            "op": "kube",
                            "parameters": {
                                "input_file_separator": ",",
                                "optimization": {
                                    "compute_pipeline_notebooks_flag": true,
                                    "daub_include_only_estimators": [
                                        "ExtraTreesClassifierEstimator",
                                        "GradientBoostingClassifierEstimator",
                                        "LGBMClassifierEstimator",
                                        "LogisticRegressionEstimator",
                                        "RandomForestClassifierEstimator",
                                 

<a id="training_connection"></a>
### Training data connection

Define connection information to COS bucket and training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/assets/historical_data/german_credit_risk/wml/german_credit_data_biased_training.csv).

**Action**: Upload training data to COS bucket and enter location information in the next cURL examples.

<a id="cos_token"></a>
### Get COS token
Retrieve COS token for further authentication calls.

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" 
target="_blank" rel="noopener no referrer">Retrieve COS authentication token</a>

In [7]:
%%bash --out cos_token

curl -s -X "POST" "$AUTH_ENDPOINT" \
     -H 'Accept: application/json' \
     -H 'Content-Type: application/x-www-form-urlencoded' \
     --data-urlencode "apikey=$COS_API_KEY" \
     --data-urlencode "response_type=cloud_iam" \
     --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
     | cut -d '"' -f 4

In [8]:
%env COS_TOKEN=$cos_token

env: COS_TOKEN=eyJraWQiOiIyMDIwMDgyMjE2NTYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLWExNDFmNWNlLWVlZmYtNDU5Ny1iZTgyLTg3MTA1NTBhMjdjMyIsImlkIjoiaWFtLVNlcnZpY2VJZC1hMTQxZjVjZS1lZWZmLTQ1OTctYmU4Mi04NzEwNTUwYTI3YzMiLCJyZWFsbWlkIjoiaWFtIiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC1hMTQxZjVjZS1lZWZmLTQ1OTctYmU4Mi04NzEwNTUwYTI3YzMiLCJuYW1lIjoiV0RQLVByb2plY3QtTWFuYWdlbWVudC1jODcxMGM1YS00YjM4LTRlMGYtYWQ3NC03MDM4Yzk4MTJjYzYiLCJzdWIiOiJTZXJ2aWNlSWQtYTE0MWY1Y2UtZWVmZi00NTk3LWJlODItODcxMDU1MGEyN2MzIiwic3ViX3R5cGUiOiJTZXJ2aWNlSWQiLCJhY2NvdW50Ijp7InZhbGlkIjp0cnVlLCJic3MiOiIxY2Y5ZjBlYjZhZjQ0NTlmOTQ3ODcxYmQzM2I0NzMxOCIsImZyb3plbiI6dHJ1ZX0sImlhdCI6MTU5ODM0NTYyNywiZXhwIjoxNTk4MzQ5MjI3LCJpc3MiOiJodHRwczovL2lhbS50ZXN0LmNsb3VkLmlibS5jb20vb2lkYy90b2tlbiIsImdyYW50X3R5cGUiOiJ1cm46aWJtOnBhcmFtczpvYXV0aDpncmFudC10eXBlOmFwaWtleSIsInNjb3BlIjoiaWJtIG9wZW5pZCIsImNsaWVudF9pZCI6ImRlZmF1bHQiLCJhY3IiOjEsImFtciI6WyJwd2QiXX0.jAPtBxNmAQsUrmdeDog87dff-JfFQWRkBNHklcMOzezpiVmlavpzklvEpwPI2N261g0vb05Fk9yI6Q9IUHsbc6KulD-kcfcI

<a id="cos_upload"></a>
### Upload file to COS
Upload your local dataset into your COS bucket

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-put-object" 
target="_blank" rel="noopener no referrer">Upload file to COS</a>

In [9]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $COS_TOKEN" \
    --header "Content-Type: application/octet-stream" \
    --data-binary "@../../../../data/credit_risk/credit_risk_training_light.csv" \
    "$COS_ENDPOINT/$COS_BUCKET/credit_risk_training_light.csv"

There should be an empty response when upload finished succesfully.

<a id="run"></a>
## 3. Experiment run

This section provides samples about how to trigger AutoAI experiment via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_create" 
target="_blank" rel="noopener no referrer">Schedule a training job for AutoAI experiment</a>

In [10]:
%%bash --out training_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data  '{"space_id": "'"$SPACE_ID"'", "training_data_references": [{"type": "s3", "id": "credit_risk_training_light.csv", "connection": {"endpoint_url": "'"$COS_ENDPOINT"'", "access_key_id": "'"$COS_ACCESS_KEY_ID"'", "secret_access_key": "'"$COS_SECRET_ACCESS_KEY"'"}, "location": {"bucket": "'"$COS_BUCKET"'", "path": "credit_risk_training_light.csv"}}], "results_reference": {"type": "s3", "id": "autoai_results", "connection": {"endpoint_url": "'"$COS_ENDPOINT"'", "access_key_id": "'"$COS_ACCESS_KEY_ID"'", "secret_access_key": "'"$COS_SECRET_ACCESS_KEY"'"}, "location": {"bucket": "'"$COS_BUCKET"'", "path": "."}}, "tags": [{"value": "autoai"}], "pipeline": {"href": "/v4/pipelines/'"$PIPELINE_ID"'"}}' \
    "$WML_ENDPOINT_URL/ml/v4/trainings?version=2020-08-01" | awk -F'"id":' '{print $2}' | cut -c2-37

In [11]:
%env TRAINING_ID=$training_id

env: TRAINING_ID=9acce4bc-ec03-4e5d-b2b1-c897b456fb6d


<a id="training_details"></a>
### Get training details
Training is an asynchronous endpoint. In case you want to monitor training status and details,
you need to use a GET method and specify which training you want to monitor by usage of training ID.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_get" 
target="_blank" rel="noopener no referrer">Get information about training job</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

### Get training status

In [26]:
%%bash

STATUS=$(curl -sk -X GET\
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
STATUS=${STATUS#*state\":\"}
STATUS=${STATUS%%\"*}
echo $STATUS

completed


Please make sure that training is completed before you go to the next sections.
Monitor `state` of your training by running above cell couple of times.

<a id="download_model"></a>
### Get selected pipeline model

Download a scikit-learn pipeline model object from the
AutoAI training job.
As all of the trained pipeline models are stored in specified user COS bucket, you can download selected model
via COS UI or via following cURL call.
Please have in mind, that you need to specify COS bucket and model location in that bucket.
This information can be retrieved from the previous call output (GET treining) under:
- entity -- status -- metrics  -- specific list entry -- intermediate_model -- location -- model

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-get-object" 
target="_blank" rel="noopener no referrer">Download file from COS</a>

In [27]:
%%bash --out model_path

PATH=$(curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")


PATH=${PATH#*model\":\"}
MODEL_PATH=${PATH%%\"*}
echo $MODEL_PATH

In [28]:
%env MODEL_PATH=$model_path

env: MODEL_PATH=./9acce4bc-ec03-4e5d-b2b1-c897b456fb6d/data/automl/pre_hpo_d_output/Pipeline1/model.pickle


In [29]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "model.pickle" \
    "$COS_ENDPOINT/$COS_BUCKET/$MODEL_PATH"

If everything went right, you should be able to see your model under specified local path.

In [30]:
!ls -l model.pickle

-rw-r--r--  1 jansoltysik  staff  106522 Aug 25 11:21 model.pickle


<a id="runs"></a>
## 4. Historical runs

In this section you will see cURL examples describing how to get historical training runs information.

Output should be similar to the output from training creation but you should see more trainings entries.  
Listing trainings:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_list" 
target="_blank" rel="noopener no referrer">Get list of historical training jobs information</a>

In [ ]:
%%bash

HISTORICAL_TRAINING_LIMIT_TO_GET=2

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings?space_id=$SPACE_ID&version=2020-08-01&limit=$HISTORICAL_TRAINING_LIMIT_TO_GET" \
    | python -m json.tool

<a id="training_cancel"></a>
### Cancel training run

**Tip:** If you want to cancel your training, please convert below cell to `code`, specify training ID and run.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Canceling training</a>

---

<a id="deploy_and_score"></a>
## 5. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

Before deployment creation, you need store your model in WML repository.
Please see below cURL call example how to do it. Remember that you need to
specify where your chosen model is stored in COS.

<a id="files_preparation"></a>
### Files preparation

Download all needed files (model.pickle, pipeline-model.json and schema.json) from COS.  
<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-get-object" 
target="_blank" rel="noopener no referrer">Download file from COS</a>

In [32]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "model.pickle" \
    "$COS_ENDPOINT/$COS_BUCKET/$MODEL_PATH"

Along with pipeline model, there is also a json file stored with experiment information.  
It will be needed during model repository store action.

In [33]:
%%bash --out pipeline_model_json_path

PATH=$(curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
PATH=${PATH#*pipeline_model\":\"}
PIPELINE_MODEL_PATH=${PATH%%\"*}
echo $PIPELINE_MODEL_PATH

In [34]:
%env PIPELINE_MODEL_JSON_PATH=$pipeline_model_json_path

env: PIPELINE_MODEL_JSON_PATH=./9acce4bc-ec03-4e5d-b2b1-c897b456fb6d/data/automl/pre_hpo_d_output/Pipeline1/pipeline-model.json


In [35]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "pipeline-model.json" \
    "$COS_ENDPOINT/$COS_BUCKET/$PIPELINE_MODEL_JSON_PATH"

In [36]:
%%bash

tar -czvf model.pickle.tar.gz model.pickle

a model.pickle


Prepare compressed file with model and experiment information.

In [42]:
!ls -l model*

-rw-r--r--  1 jansoltysik  staff  106522 Aug 25 11:21 model.pickle
-rw-r--r--  1 jansoltysik  staff   27480 Aug 25 11:21 model.pickle.tar.gz
-rw-r--r--  1 jansoltysik  staff     297 Aug 25 11:14 model.tar.gz


<a id="model_store"></a>
### Store AutoAI model

Store information about your model to WML repository.
You can take model schema from downloaded `schema.json` from COS.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [38]:
%%bash --out model_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data '{"space_id": "'"$SPACE_ID"'","name": "autoai_credit_risk_model","description": "This is description","type": "wml-hybrid_0.1", "software_spec": {"name": "hybrid_0.1"}}' \
    "$WML_ENDPOINT_URL/ml/v4/models?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [39]:
%env MODEL_ID=$model_id

env: MODEL_ID=586e4926-35b8-4f74-9bcd-d3d9c271b7da


<a id="model_content_upload"></a>
### Upload model content

Now you need to upload your model content into the WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_upload_content" 
target="_blank" rel="noopener no referrer">Upload model content</a>

In [40]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data-binary "@pipeline-model.json" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "f1766fab-6019-4c4f-a014-5d0878041383",
    "content_format": "native",
    "persisted": true
}


In [43]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@model.pickle.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/content?version=2020-08-01&space_id=$SPACE_ID&content_format=pipeline-node&pipeline_node_id=automl" \
    | python -m json.tool

{
    "attachment_id": "f25fb5c5-3962-4047-8bd9-537b465b77d3",
    "content_format": "pipeline-node",
    "persisted": true,
    "pipeline_node_id": "automl"
}


<a id="model_content_download"></a>
### Download model content

If you want to download your saved model, please make the following call.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_filtered_download" 
target="_blank" rel="noopener no referrer">Download model content</a>

In [44]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$WML_ENDPOINT_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [45]:
!ls -l model.tar.gz

-rw-r--r--  1 jansoltysik  staff  297 Aug 25 11:23 model.tar.gz


## <a id="deployment_creation"></a>
### Deployment creation

An AutoAI Batch deployment creation is presented below.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [46]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data '{"space_id": "'"$SPACE_ID"'","name": "AutoAI deployment","description": "This is description","batch": {}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}],"asset": {"id": "'"$MODEL_ID"'"}}' \
    "$WML_ENDPOINT_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [47]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=4e21a2d6-f823-4b2f-bbf7-095c38b135a0


<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [48]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "586e4926-35b8-4f74-9bcd-d3d9c271b7da"
        },
        "batch": {},
        "custom": {},
        "description": "This is description",
        "hybrid_pipeline_hardware_specs": [
            {
                "hardware_spec": {
                    "name": "M"
                },
                "node_runtime_id": "auto_ai.kb"
            }
        ],
        "name": "AutoAI deployment",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750",
        "status": {
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2020-08-25T09:23:34.852Z",
        "description": "This is description",
        "id": "4e21a2d6-f823-4b2f-bbf7-095c38b135a0",
        "modified_at": "2020-08-25T09:23:34.852Z",
        "name": "AutoAI deployment",
        "owner": "IBMid-5500067NJD",
        "space_id": "74133c06-dce2-4dfc-b913-2e0dc8efc750"
    }
}


<a id="batch_score"></a>
### Score your Batch deployment
Scoring for Batch deployment is done by creating `jobs`. User can specify job payload as a json or as data connection to eg. COS.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_create" 
target="_blank" rel="noopener no referrer">Create deployment job</a>

In [49]:
%%bash --out job_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data '{"name": "AutoAI job", "space_id": "'"$SPACE_ID"'","deployment": {"id": "'"$DEPLOYMENT_ID"'"}, "hybrid_pipeline_hardware_specs": [{"node_runtime_id": "auto_ai.kb", "hardware_spec": {"name": "M"}}], "scoring": {"input_data": [{"fields": ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings", "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration", "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents", "Telephone", "ForeignWorker"], "values": [["less_0", 6, "all_credits_paid_back", "car_used", 250, "less_100", "1_to_4", 2, "male", "none", 2, "savings_insurance", 28, "stores", "rent", 1, "skilled", 1, "none", "yes"]]}]}}' \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [50]:
%env JOB_ID=$job_id

env: JOB_ID=4dfc06c2-b7eb-430a-ad8c-cd2321478cdc


<a id="job_list"></a>
### Listing all Batch jobs

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_list" 
target="_blank" rel="noopener no referrer">List jobs</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="job_get"></a>
### Get particular job details

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_get" 
target="_blank" rel="noopener no referrer">Get job details</a>

In [56]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployment_jobs/$JOB_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "deployment": {
            "id": "4e21a2d6-f823-4b2f-bbf7-095c38b135a0"
        },
        "scoring": {
            "input_data": [
                {
                    "fields": [
                        "CheckingStatus",
                        "LoanDuration",
                        "CreditHistory",
                        "LoanPurpose",
                        "LoanAmount",
                        "ExistingSavings",
                        "EmploymentDuration",
                        "InstallmentPercent",
                        "Sex",
                        "OthersOnLoan",
                        "CurrentResidenceDuration",
                        "OwnsProperty",
                        "Age",
                        "InstallmentPlans",
                        "Housing",
                        "ExistingCreditsCount",
                        "Job",
                        "Dependents",
                        "Telephone",
                        "Fore

<a id="job_cancel"></a>
### Cancel job

**Tip:** You can cancel running job by calling delete method.
Just convert below cell to `code` and run it.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_delete" 
target="_blank" rel="noopener no referrer">Cancel job</a>

<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$WML_ENDPOINT_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="training_delete"></a>
### Delete training run
**Tip:** You can completely delete a training run with its metadata.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Deleting training</a>

<a id="job_delete"></a>
### Delete job

**Tip:** If you want remove job completely (with metadata), just specify `hard_delete` to True.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_delete" 
target="_blank" rel="noopener no referrer">Delete job</a>

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a AutoAI model in WML. 


### Authors

**Amadeusz Masny**, Python Software Developer in Watson Machine Learning at IBM  
**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.